In [42]:
import json
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
import time
from tqdm.auto import tqdm

In [43]:
from retrying import retry
import openai

@retry(stop_max_attempt_number=5, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def generate(prompt, history=[]):

    client = openai.OpenAI(api_key="sk-xxx", base_url="xxx")
    completion = client.chat.completions.create(
        model="xxx",
        messages = [{"role": "user", "content": prompt}],
        temperature = 0.
    )
    return completion.choices[0].message.content

In [44]:
with open("../../data/align/Align_Minos.json", 'r', encoding='utf-8-sig') as f:
    datas = json.loads(f.read())


In [45]:
input_data = []
for i in datas:
    tmp = {}
    tmp['id'] = i['id']
    tmp['question'] = i['question']
    tmp['answer'] = i['answer']
    tmp['response'] = i['response']
    tmp['rank'] = i['rank']
    tmp['label'] = i['label']
    input_data.append(tmp)

In [46]:
prompt = '''判断输入的问题是否为事实性问题。
事实性问题指的是可以通过查找客观信息、逻辑推导或基于明确规律得出确定答案的问题。若是事实性问题，请输出“是”；若是非事实性问题，请输出“否”。

注意以下判断标准：
1. 具有明确规律的任务：若问题要求基于特定规则或规律执行操作（如反转句子、计算答案），且答案确定，则视为事实性问题。
2. 推理与归纳问题：若问题要求基于给定信息推导出唯一合理答案，也应视为事实性问题。
3. 开放性指令型问题：若问题没有明确答案，或仅涉及开放性任务（如创作），则视为非事实性问题。

我将给你几个示例，请进行推理、输出推理过程并用 JSON 格式输出答案，如：
推理结果：xxxxxx。
{
    "答案": xxx,
}

示例1:
问题：请给出2句唐代诗，并且分别给出作者和诗名。要求诗句中出现词语“扬州”
答案：否

示例2:
问题：请将“知不可乎骤得，托遗响于悲风”翻译为现代中文
答案：否

示例3:
问题：盘子里有猕猴桃、香蕉、柠檬、橘子四种水果若干。小刚、小林、小红、小华从中各拿了一个水果。小刚说：“小林拿了橘子。”小林说：“没有人拿香蕉。”小华说：“小红没有拿柠檬。”小红说：“小林拿了香蕉。”如果四个人当中,只有一个人说的是真话,那么以下哪项一定为真？A.小林拿了橘子B.小红拿了柠檬C.小刚拿了香蕉D.小华拿了猕猴桃
答案：是

示例4:
问题："将下面的句子反过来输出：那只美丽的啄木鸟有着长长的脖子"
答案：是

示例5:
问题：“小明因为贪玩不吃饭，妈妈就假装打玩偶，边打边说“再不好好吃饭就滚出去”，然后小明看到默默走向了餐桌。”哪个成语能最好地总结这段话:1.时不我待 2.杀鸡做猴 3.知错能改 4.自知之明。”
答案：是

问题：{input_question}
'''

In [47]:
import re

def format_check(input):
    if isinstance(input,dict) and set(list(input)) == set(['答案']):
        return True
    else:
        return False


# 定义一个函数，接受一个 markdown 字符串作为参数
def find_dicts(markdown):
    # 定义一个空列表，用于存储找到的 dict
    dicts = []
    
    if markdown == None:
        return dicts

    # 定义一个正则表达式，匹配 dict 的格式
    pattern = r"\{[^{}]*\}"
    # 使用 re.findall 方法，找出 markdown 字符串中所有匹配的子串
    matches = re.findall(pattern, markdown)
    # 遍历每个匹配的子串
    for match in matches:
        # 尝试将子串转换为 dict 类型，如果成功则添加到列表中
        try:
            d = eval(match)
            if isinstance(d, dict):
                dicts.append(d)
        except:
            # 如果转换失败，忽略该子串
            pass
    # 返回找到的 dict 列表
    return dicts

In [48]:
import traceback
import os
import json

import traceback
def deal(item,file_path):
    max_try_retry = 10
    try_num = 0
    content = prompt.replace('{input_question}',item['question'])
    # print(content)
    
    for _ in range(max_try_retry):
        try:
            response = generate(content)
            try:
                tmps = json.loads(response)
                # print('yresss')
            except:
                tmps = find_dicts(response)[0]
            
            if format_check(tmps):
                item['res'] = tmps
                with open(file_path, "a+", encoding="utf8") as f:
                    f.write(json.dumps(item, ensure_ascii=False) + "\n")
                    f.flush()
                break

        except Exception as e:
            str_e = str(e)
            if try_num== max_try_retry:
                break

            if 'InvalidRequestError' in str_e:
                if 'maximum context' in str_e:
                    break
                try_num += 1
                continue
            else:
                traceback.print_exc()


In [ ]:
file_path = "../../data/align/result/align_llm_label.jsonl"

if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        pass

with open(file_path,'r',encoding='utf-8') as f:
    L = f.readlines()
finish_id = [json.loads(i)['id'] for i in L]

input_list = [i for i in input_data if i['id'] not in finish_id]

from tqdm.contrib import tzip
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib

num_worker = 32

with tqdm_joblib(desc="My calculation", total=len(input_list)) as progress_bar:
    Parallel(n_jobs=num_worker,prefer="threads")([delayed(deal)(x,file_path=file_path) for x in input_list])

In [ ]:
count = 0
total_count = 0
import json
with open(file_path, 'r', encoding='utf-8-sig') as f:
    for line in f:
        total_count += 1
        data = json.loads(line)
        label = data['res']['答案']
        if label == '是' and data['label'] == 1:
            count += 1
        elif label == '否' and data['label'] == 0:
            count += 1
print("ACC：")
print(count/total_count)